## Library Imports

In [36]:
import pandas as pd
import geopandas as gpd
import folium
from folium import Choropleth
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import warnings
warnings.filterwarnings("ignore")

print("Libraries loaded!\n")

Libraries loaded!



## Loading Data

In [37]:
# STEP 1: LOADING DATA FROM KAGGLE INPUT
print("Loading datasets from /kaggle/input/dataset/ ...")

df_terr = pd.read_csv("/kaggle/input/parisdataset/terrasses-autorisations.csv", sep=";", low_memory=False, encoding='utf-8')
df_sem  = pd.read_csv("/kaggle/input/parisdataset/commerces-semaest.csv", sep=";", low_memory=False, encoding='utf-8')
df_arr  = pd.read_csv("/kaggle/input/parisdataset/arrondissements.csv", sep=";", low_memory=False, encoding='utf-8')

print(f"✓ Terrace authorizations: {len(df_terr):,} records")
print(f"✓ SEMAEST shops: {len(df_sem):,} shops")
print(f"✓ Arrondissements CSV: {len(df_arr)} rows\n")

Loading datasets from /kaggle/input/dataset/ ...
✓ Terrace authorizations: 23,911 records
✓ SEMAEST shops: 311 shops
✓ Arrondissements CSV: 20 rows



## Converting Arrondissements CSV To GeoDataFrame

In [38]:
# CELL 3: CREATE ARRONDISSEMENT BOUNDARIES FROM CSV
print("Creating arrondissement polygons from arrondissements.csv...")

import json
from shapely.geometry import shape

# arrondissements.csv has a column "Geometry" with full GeoJSON polygon
def parse_geometry(geojson_str):
    try:
        geom = json.loads(geojson_str)
        return shape(geom)
    except:
        return None

# Apply to actual column name
df_arr['geometry'] = df_arr['Geometry'].apply(parse_geometry)

# Create GeoDataFrame
gdf_arr = gpd.GeoDataFrame(df_arr, geometry='geometry', crs="EPSG:4326")

# Standardize column names
gdf_arr = gdf_arr.rename(columns={
    'Numéro d’arrondissement INSEE': 'c_arinsee',
    'Nom officiel de l’arrondissement': 'l_ar',
    'Surface': 'surface'
})

gdf_arr = gdf_arr[['c_arinsee', 'l_ar', 'surface', 'geometry']].copy()
gdf_arr['c_arinsee'] = gdf_arr['c_arinsee'].astype(str)
gdf_arr['surface'] = pd.to_numeric(gdf_arr['surface'], errors='coerce')

print(f"20 arrondissements loaded with polygon geometry from CSV!")

Creating arrondissement polygons from arrondissements.csv...
20 arrondissements loaded with polygon geometry from CSV!


## Assigning Terrasses To Arrondissements

In [39]:
# STEP 3: ASSIGN TERRASSES TO ARRONDISSEMENTS
print("Mapping terraces to arrondissements...")

coords = df_terr['geo_point_2d'].str.split(', ', expand=True)

gdf_terr = gpd.GeoDataFrame(
    df_terr,
    geometry=gpd.points_from_xy(
        coords[1].astype(float),
        coords[0].astype(float)
    ),
    crs="EPSG:4326"
)

# Spatial join with arrondissement polygons
gdf_terr_joined = gdf_terr.sjoin(gdf_arr[['c_arinsee', 'l_ar', 'geometry']], how='left', predicate='within')

# Count terraces per arrondissement
terr_count = gdf_terr_joined['c_arinsee'].value_counts().reset_index()
terr_count.columns = ['c_arinsee', 'terrace_count']
terr_count['c_arinsee'] = terr_count['c_arinsee'].astype(str)

print(f"{len(terr_count)} arrondissements have terraces")
print("Terrace mapping complete!\n")

Mapping terraces to arrondissements...
20 arrondissements have terraces
Terrace mapping complete!



## Assigning SEMAEST Shops To Arrondissements

In [40]:
# STEP 4: ASSIGN SEMAEST SHOPS TO ARRONDISSEMENTS
print("Mapping SEMAEST shops to arrondissements using LAT_ok & LONG_ok...")

# Using the coordinate columns
df_sem['lat'] = pd.to_numeric(df_sem['LAT_ok'], errors='coerce')
df_sem['lon'] = pd.to_numeric(df_sem['LONG_ok'], errors='coerce')

# Drop rows with missing coordinates
df_sem_clean = df_sem.dropna(subset=['lat', 'lon']).copy()

# Create GeoDataFrame
gdf_sem = gpd.GeoDataFrame(
    df_sem_clean,
    geometry=gpd.points_from_xy(df_sem_clean['lon'], df_sem_clean['lat']),
    crs="EPSG:4326"
)

# Spatial join with arrondissement polygons
gdf_sem_joined = gdf_sem.sjoin(gdf_arr[['c_arinsee', 'l_ar', 'geometry']], how='left', predicate='within')

# Count SEMAEST shops per arrondissement
sem_count = gdf_sem_joined['c_arinsee'].value_counts().reset_index()
sem_count.columns = ['c_arinsee', 'semaest_count']
sem_count['c_arinsee'] = sem_count['c_arinsee'].astype(str)

print(f"{len(sem_count)} arrondissements have SEMAEST shops")
print("SEMAEST mapping complete!\n")

Mapping SEMAEST shops to arrondissements using LAT_ok & LONG_ok...
16 arrondissements have SEMAEST shops
SEMAEST mapping complete!



## Build Final Ranking & Commercial Activity Index

In [41]:
# STEP 5: BUILD FINAL RANKING & COMMERCIAL ACTIVITY INDEX
print("Computing Commercial Activity Index...")

# Base table
base = gdf_arr[['c_arinsee', 'l_ar', 'surface']].copy()
base['area_km2'] = base['surface'] / 1_000_000

# Merge counts
data = base.merge(terr_count, on='c_arinsee', how='left').fillna(0)
data = data.merge(sem_count, on='c_arinsee', how='left').fillna(0)

# Density
data['terrace_per_km2'] = data['terrace_count'] / data['area_km2']
data['semaest_per_km2'] = data['semaest_count'] / data['area_km2']

# Normalize & Final Score (0–100)
scaler = MinMaxScaler()
data[['t_norm', 's_norm']] = scaler.fit_transform(data[['terrace_per_km2', 'semaest_per_km2']])
data['Commercial_Score'] = (data['t_norm'] + data['s_norm']) * 50

# Final ranking
data = data.sort_values('Commercial_Score', ascending=False).reset_index(drop=True)
data['Rank'] = data.index + 1

print("ANALYSIS COMPLETE!\n")
print("TOP 10 BEST ARRONDISSEMENTS FOR E-COMMERCE EXPANSION:")
display(data[['Rank', 'l_ar', 'terrace_count', 'semaest_count', 'terrace_per_km2', 'Commercial_Score']].head(10))

Computing Commercial Activity Index...
ANALYSIS COMPLETE!

TOP 10 BEST ARRONDISSEMENTS FOR E-COMMERCE EXPANSION:


,Rank,l_ar,terrace_count,semaest_count,terrace_per_km2,Commercial_Score
0,1,Luxembourg,778,42.0,361.340205,72.894014
1,2,Temple,610,16.0,520.974418,70.882636
2,3,Entrepôt,1539,37.0,532.205626,69.565062
3,4,Popincourt,1730,36.0,471.975879,57.052318
4,5,Panthéon,1320,17.0,519.813023,52.921888
5,6,Bourse,689,1.0,695.149470,52.586086
6,7,Hôtel-de-Ville,899,6.0,561.669418,48.769667
7,8,Opéra,1357,1.0,622.961924,45.314925
8,9,Louvre,856,4.0,469.140615,37.266809
9,10,Buttes-Montmartre,1532,27.0,255.501483,25.841733


## VISUALIZATION 1: Terrace Density Map

In [42]:
# VISUALIZATION 1: Terrace Density Map
print("Generating Map: Foot Traffic (Terrace Density)")
m1 = folium.Map(location=[48.8566, 2.3522], zoom_start=12, tiles="CartoDB positron")

Choropleth(
    geo_data=gdf_arr.__geo_interface__,
    data=data,
    columns=['c_arinsee', 'terrace_per_km2'],
    key_on='feature.properties.c_arinsee',
    fill_color='OrRd',
    legend_name='Terraces per km²',
    highlight=True
).add_to(m1)

m1.save("Terrace_Density_Map.html")
m1

Generating Map: Foot Traffic (Terrace Density)


## VISUALIZATION 2: SEMAEST Investment Map

In [43]:
# VISUALIZATION 2: SEMAEST Investment Map
print("Generating Map: City Retail Investment (SEMAEST)")

m2 = folium.Map(location=[48.8566, 2.3522], zoom_start=12, tiles="CartoDB positron")

for _, r in gdf_sem_joined.iterrows():
    folium.CircleMarker(
        location=[r.geometry.y, r.geometry.x],
        radius=6,
        color="purple",
        weight=2,
        fill=True,
        fillOpacity=0.8,
        popup=folium.Popup(
            f"<b>{r.get('Enseigne', 'SEMAEST Shop')}</b><br>{r['Adresse']}<br><i>{r['l_ar']}</i>",
            max_width=300
        )
    ).add_to(m2)

m2.save("SEMAEST_Shops_Map.html")
m2

Generating Map: City Retail Investment (SEMAEST)


## VISUALIZATION 3: Final Commercial Score Map

In [44]:
# VISUALIZATION 3: Final Commercial Score Map
print("Generating Map: Final Commercial Activity Index")
m3 = folium.Map(location=[48.8566, 2.3522], zoom_start=12, tiles="CartoDB positron")

Choropleth(
    geo_data=gdf_arr.__geo_interface__,
    data=data,
    columns=['c_arinsee', 'Commercial_Score'],
    key_on='feature.properties.c_arinsee',
    fill_color='YlGnBu',
    legend_name='Commercial Activity Score (0–100)',
).add_to(m3)

m3.save("Final_Commercial_Score_Map.html")
m3

Generating Map: Final Commercial Activity Index


## CHART 1: Top 10 by Foot Traffic

In [45]:
# CHART 1: Top 10 by Foot Traffic
px.bar(data.head(10), x='l_ar', y='terrace_per_km2',
       title="Top 10: Highest Foot Traffic (Terraces per km²)",
       labels={'l_ar': 'Arrondissement', 'terrace_per_km2': 'Terraces per km²'},
       color='terrace_per_km2', color_continuous_scale='Reds').show()

## CHART 2: Top 10 by City Investment

In [46]:
# CHART 2: Top 10 by City Investment
px.bar(
    data.head(10),
    x='l_ar',
    y='semaest_count',
    title="Top 10 Arrondissements by City Retail Investment (SEMAEST Shops)",
    labels={
        'l_ar': 'Arrondissement',
        'semaest_count': 'Number of SEMAEST-backed shops'
    },
    color='semaest_count',
    color_continuous_scale='Purples',
    text='semaest_count',
    height=600
).update_layout(
    xaxis_title="Arrondissement",
    yaxis_title="Number of SEMAEST shops",
    showlegend=False
).show()

## CHART 3: Radar Chart – Top 5 Comparison

In [47]:
# CHART 3: Radar Chart – Top 5 Comparison
top5 = data.head(5).copy()

# Normalize the two big metrics to match the 0–100 scale of the score
max_terrace = top5['terrace_per_km2'].max()
max_semaest = top5['semaest_per_km2'].max()

top5['Foot Traffic (scaled)'] = (top5['terrace_per_km2'] / max_terrace) * 100
top5['City Investment (scaled)'] = (top5['semaest_per_km2'] / max_semaest) * 100

fig = go.Figure()

for _, row in top5.iterrows():
    fig.add_trace(go.Scatterpolar(
        r=[
            row['Foot Traffic (scaled)'],
            row['City Investment (scaled)'],
            row['Commercial_Score']
        ],
        theta=['Foot Traffic', 'City Investment', 'Overall Score'],
        fill='toself',
        name=row['l_ar']
    ))

fig.update_layout(
    title="Top 5 Arrondissements – Commercial DNA Comparison",
    polar=dict(
        radialaxis=dict(visible=True, range=[0, 100], tickfont_size=12)
    ),
    showlegend=True,
    height=650
)

fig.show()

## Final Executive Summary

In [48]:
# FINAL EXECUTIVE SUMMARY
print("\n" + "="*80)
print("E-COMMERCE EXPANSION RECOMMENDATIONS – PARIS 2025")
print("="*80)
print(f"Best overall:       {data.iloc[0]['l_ar']} (Score: {data.iloc[0]['Commercial_Score']:.1f}/100)")
print(f"Runner-up:          {data.iloc[1]['l_ar']} (Score: {data.iloc[1]['Commercial_Score']:.1f}/100)")
print(f"High-growth zones:   {data.iloc[2]['l_ar']}, {data.iloc[3]['l_ar']}, {data.iloc[4]['l_ar']}")
print("\nStrategic Recommendations:")
print("• Flagship stores & premium pop-ups → 1er, 2e, 6e")
print("• Local delivery + omnichannel hubs → 10e, 11e, 18e")
print("• Emerging creative & affordable zones → 19e, 20e, 13e")
print("• Avoid for physical expansion → 12e, 15e, 16e (lowest scores)")
print("="*80)
print("Analysis based on 23,911 terraces + 311 SEMAEST shops – 100% from open data paris")
print("="*80)


E-COMMERCE EXPANSION RECOMMENDATIONS – PARIS 2025
Best overall:       Luxembourg (Score: 72.9/100)
Runner-up:          Temple (Score: 70.9/100)
High-growth zones:   Entrepôt, Popincourt, Panthéon

Strategic Recommendations:
• Flagship stores & premium pop-ups → 1er, 2e, 6e
• Local delivery + omnichannel hubs → 10e, 11e, 18e
• Emerging creative & affordable zones → 19e, 20e, 13e
• Avoid for physical expansion → 12e, 15e, 16e (lowest scores)
Analysis based on 23,911 terraces + 311 SEMAEST shops – 100% from open data paris


In [50]:
# SAVE CLEAN TABLES FOR POWER BI / TABLEAU / EXCEL – ONE CLICK
print("Saving clean tables for Power BI...")

# 1. Final ranking with everything (the star table)
data.to_csv("Paris_Commercial_Score_Ranking_2025.csv", index=False, encoding='utf-8')
print("→ Paris_Commercial_Score_Ranking_2025.csv")

# 2. Terraces with arrondissement (for detailed maps)
gdf_terr_joined[['c_arinsee', 'l_ar', 'geometry']].dropna().to_csv("Terraces_by_Arrondissement.csv", index=False, encoding='utf-8')
print("→ Terraces_by_Arrondissement.csv")

# 3. SEMAEST shops with coordinates and arrondissement
gdf_sem_joined.to_csv("SEMAEST_Shops_with_Arrondissement.csv", index=False, encoding='utf-8')
print("→ SEMAEST_Shops_with_Arrondissement.csv")

# 4. Arrondissements with geometry (for perfect maps in Power BI)
gdf_arr.to_csv("Paris_Arrondissements_Clean.csv", index=False, encoding='utf-8')
print("→ Paris_Arrondissements_Clean.csv")

print("\nAll files saved! Download them now → right-click in the file panel → Download")

Saving clean tables for Power BI...
→ Paris_Commercial_Score_Ranking_2025.csv
→ Terraces_by_Arrondissement.csv
→ SEMAEST_Shops_with_Arrondissement.csv
→ Paris_Arrondissements_Clean.csv

All files saved! Download them now → right-click in the file panel → Download
